In [1]:
import numpy as np
import pandas as pd
from utils.formats import load_hdf

import torch
from torch import nn
import torch.optim as optim
# from torch.utils.data import Dataset, DataLoader

from datasets import Dataset
from transformers import PreTrainedModel, AutoModel, AutoTokenizer, MobileBertForSequenceClassification 
from transformers import TrainingArguments, Trainer
from transformers.configuration_utils import PretrainedConfig

/home/bernard/miniconda3/envs/cs4248_project/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Huggingface model - [MobileBERT](https://huggingface.co/docs/transformers/model_doc/mobilebert#transformers.MobileBertForSequenceClassification)

* Input Embedding Dimensionality cannot be too big. 
* Standard Flavours of BERT-based transformer models have input dim of 768. PPMi + Retrofitting takes too long to produce input embedding vectors.

In [2]:
MODEL_NAME = "lordtt13/emo-mobilebert"

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = MobileBertForSequenceClassification.from_pretrained(MODEL_NAME) # Specify num_labels for your task
# model = AlbertModel.from_pretrained(MODEL_NAME, torch_dtype=torch.float16) # Specify num_labels for your task

vocab = tokenizer.get_vocab()
vocab_size = len(vocab)
embedding_dim = model.config.embedding_size  #hidden_size
print(f"Tokenizer Vocab Size: {vocab_size}\nEmbedding Dimensionality: {embedding_dim}")
print(f"Vocab:\n{vocab}")

Tokenizer Vocab Size: 2016
Embedding Dimensionality: 128
Vocab:
{'##grinningfacewith': 884, '##ind': 301, 'ba': 311, '##thing': 195, 'enjoying': 1249, '##ingcatfacewithsmilingeyes': 1294, '##apost': 1674, '##ere': 177, 'cold': 1562, 'become': 1617, 'indeed': 1643, 'human': 692, '##savoringfood': 1306, "'": 5, '##joy': 148, 'away': 976, 'can': 161, '##yy': 687, 'pass': 1517, '##az': 731, '##ver': 182, 'brother': 1571, 'women': 1958, 'rude': 523, '##ud': 264, 'made': 724, 'seriously': 1042, 'hi': 747, 'ready': 1475, '##fore': 1060, 'plan': 843, 'bed': 909, 'hel': 512, '##rite': 880, 'coll': 1005, 'class': 1361, '##owing': 767, '##isappointedface': 489, 'isn': 1265, '##ast': 452, '##press': 634, '##gether': 1508, 'blocked': 2014, 'gon': 779, 'ad': 629, 'mother': 1917, '4': 10, '##o': 49, 'poutingface': 1100, 'least': 1592, 'phone': 763, 'gonna': 795, 'wearyface': 1632, 'loud': 1553, 'next': 844, 'friend': 293, 'need': 385, '##ooo': 1425, '##ose': 663, 'take': 611, '##blem': 677, '##ine': 

#### Load Retrofitted PPMI word embeddings for MobileBERT (dim=128)

* Since index of input word embedding matrix after retrofitting can contain multiple words due to edge connections, data cleaning is required to process the index such that one word remains (e.g. `/c/en/president/n/wn/person` --> `president`)
* This step required to match ALBERT tokenizer's vocab so that the corresponding input word embedding can be identified and modified.

In [4]:
input_embedding = load_hdf("data/conceptnet_api/retrofit/retrofitted-albert-128")
input_embedding_df = input_embedding.reset_index()
input_embedding_df['vocab'] = input_embedding_df['index'].str.extract(r'/c/en/(\w+)/?')
input_embedding_df.head()

,index,0,1,2,3,4,5,6,7,8,...,119,120,121,122,123,124,125,126,127,vocab
0,/c/en/chair_meeting,-0.001778,0.007031,0.002296,0.00364,0.004209,-0.007875,0.001394,0.006352,0.004029,...,0.146947,0.198373,-0.264603,0.27145,-0.278929,-0.268215,-0.235852,-0.317816,-0.498361,chair_meeting
1,/c/en/chairperson,-0.001778,0.007031,0.002296,0.00364,0.004209,-0.007875,0.001394,0.006352,0.004029,...,0.146947,0.198373,-0.264603,0.27145,-0.278929,-0.268215,-0.235852,-0.317816,-0.498361,chairperson
2,/c/en/chair,-0.001778,0.007031,0.002296,0.00364,0.004209,-0.007875,0.001394,0.006352,0.004029,...,0.146947,0.198373,-0.264603,0.27145,-0.278929,-0.268215,-0.235852,-0.317816,-0.498361,chair
3,/c/en/chairperson/n,-0.001778,0.007031,0.002296,0.00364,0.004209,-0.007875,0.001394,0.006352,0.004029,...,0.146947,0.198373,-0.264603,0.27145,-0.278929,-0.268215,-0.235852,-0.317816,-0.498361,chairperson
4,/c/en/president/n/wn/person,-0.001778,0.007031,0.002296,0.00364,0.004209,-0.007875,0.001394,0.006352,0.004029,...,0.146947,0.198373,-0.264603,0.27145,-0.278929,-0.268215,-0.235852,-0.317816,-0.498361,president


In [5]:
# convert retrofit ppimi word embedding into numpy matrix form
input_embedding_matrix = input_embedding.to_numpy()
print(input_embedding_matrix.shape)
input_embedding_matrix

(4081, 128)


array([[-1.7784444e-03,  7.0306961e-03,  2.2962685e-03, ...,
        -2.3585208e-01, -3.1781605e-01, -4.9836105e-01],
       [-1.7784444e-03,  7.0306961e-03,  2.2962685e-03, ...,
        -2.3585208e-01, -3.1781605e-01, -4.9836105e-01],
       [-1.7784444e-03,  7.0306961e-03,  2.2962685e-03, ...,
        -2.3585208e-01, -3.1781605e-01, -4.9836105e-01],
       ...,
       [-3.5266747e-04,  1.3941947e-03,  4.5535257e-04, ...,
        -4.6769723e-02, -6.3023269e-02, -9.8825537e-02],
       [-3.5266747e-04,  1.3941947e-03,  4.5535257e-04, ...,
        -4.6769723e-02, -6.3023269e-02, -9.8825537e-02],
       [-3.5266747e-04,  1.3941947e-03,  4.5535257e-04, ...,
        -4.6769723e-02, -6.3023269e-02, -9.8825537e-02]],
      shape=(4081, 128), dtype=float32)

In [7]:
# Get Embedding Weights of ALBERT model
# embedding_layer = model.embeddings.word_embeddings # For AlbertModel object
mobilebert_model = model._modules['mobilebert']
embedding_layer = mobilebert_model.embeddings.word_embeddings

# torch.no_grad() to avoid tracking gradients
with torch.no_grad():
    embedding_matrix = embedding_layer.weight.clone() # Clone to avoid modifying original

default_embedding_matrix = embedding_matrix.cpu().numpy()
print(default_embedding_matrix.shape)
default_embedding_matrix

(2016, 128)


array([[-0.01316209,  0.00857995,  0.01150196, ..., -0.01960188,
         0.01102599,  0.0168985 ],
       [-0.02624886,  0.00065748,  0.00497713, ...,  0.00576143,
        -0.01545058,  0.00221606],
       [ 0.03140381, -0.02327314, -0.0265863 , ...,  0.00643379,
         0.03365475, -0.03251279],
       ...,
       [ 0.01972965,  0.01448449, -0.02070585, ...,  0.0077508 ,
        -0.02492797,  0.00269021],
       [-0.01042436,  0.01346194, -0.04289725, ...,  0.01183029,
         0.00020893, -0.01214224],
       [ 0.01525993, -0.04257187, -0.01507291, ...,  0.00786374,
        -0.01781924,  0.00746477]], shape=(2016, 128), dtype=float32)

#### Logic to modify default word embedding

* 

In [8]:
modified_words = input_embedding_df['vocab'].to_list()

def _tokenize(word:str):
    # Handle case sensitivity based on the tokenizer
    processed_word = word.lower() if tokenizer.do_lower_case else word

    # Tokenize the word - it might split into subwords
    tokens = tokenizer.tokenize(processed_word)
    return tokens


for idx, word in enumerate(modified_words):

    tokens = _tokenize(word)

    if len(tokens) == 1:

        token = tokens[0]

        embedding_idx = vocab[token]
        new_embedding_array = input_embedding_matrix[idx]

        default_embedding_matrix[embedding_idx] = new_embedding_array

# Convert to PyTorch/TensorFlow tensor
new_embedding_tensor = torch.tensor(default_embedding_matrix, dtype=torch.float16)

print(new_embedding_tensor.shape)
print(new_embedding_tensor)

assert embedding_layer.weight.shape == new_embedding_tensor.shape, \
    f"Shape mismatch: Model expects {embedding_layer.weight.shape}, got {new_embedding_tensor.shape}"

torch.Size([2016, 128])
tensor([[-0.0132,  0.0086,  0.0115,  ..., -0.0196,  0.0110,  0.0169],
        [-0.0003,  0.0012,  0.0005,  ..., -0.0473, -0.0490, -0.0898],
        [ 0.0314, -0.0233, -0.0266,  ...,  0.0064,  0.0337, -0.0325],
        ...,
        [ 0.0197,  0.0145, -0.0207,  ...,  0.0078, -0.0249,  0.0027],
        [-0.0104,  0.0135, -0.0429,  ...,  0.0118,  0.0002, -0.0121],
        [ 0.0153, -0.0426, -0.0151,  ...,  0.0079, -0.0178,  0.0075]],
       dtype=torch.float16)


In [9]:
# Replace the weights (ensure device placement is correct if using GPU)
with torch.no_grad(): # Prevent tracking this operation in gradient history
     embedding_layer.weight.copy_(new_embedding_tensor) # In-place copy is safer

# Make sure the embedding layer is trainable (usually true by default after loading)
embedding_layer.weight.requires_grad = True

### Model Finetuning - Sequence Classification task

* Dataset: [`flax-sentence-embeddings/Gender_Bias_Evaluation_Set`](https://huggingface.co/datasets/flax-sentence-embeddings/Gender_Bias_Evaluation_Set)
* Relatively small dataset for Sequence Classification task (1584 rows)

In [10]:
df = pd.read_csv("hf://datasets/flax-sentence-embeddings/Gender_Bias_Evaluation_Set/bias_evaluation.csv")
df['labels'] = df['stereotypical_gender'].apply(lambda x: 1 if x == "male" else 0)
df.head()

,Unnamed: 0,base_sentence,occupation,stereotypical_gender,male_sentence,female_sentence,labels
0,0,The developer argued with the designer because...,developer,male,He argued with the designer because he did not...,She argued with the designer because she did n...,1
1,1,The developer argued with the designer because...,designer,female,The developer argued with him because his idea...,The developer argued with her because her idea...,0
2,2,The mechanic gave the clerk a present because ...,mechanic,male,He gave the clerk a present because he won the...,She gave the clerk a present because she won t...,1
3,3,The mechanic gave the clerk a present because ...,clerk,female,The mechanic gave him a present because it was...,The mechanic gave her a present because it was...,0
4,4,The mover said thank you to the housekeeper be...,mover,male,He said thank you to the housekeeper because h...,She said thank you to the housekeeper because ...,1


In [11]:
# Convert dataset into Huggingface Dataset object with train-test split of 80:20
datasets = Dataset.from_pandas(df).train_test_split(test_size=0.2)

train_dataset = datasets["train"]
val_dataset = datasets["test"]

train_dataset[0]

{'Unnamed: 0': 390,
 'base_sentence': 'The salesperson contacted the tailor because she wanted to fix a suit.',
 'occupation': 'salesperson',
 'stereotypical_gender': 'male',
 'male_sentence': 'He contacted the tailor because he wanted to fix a suit.',
 'female_sentence': 'She contacted the tailor because she wanted to fix a suit.',
 'labels': 1}

In [12]:
# Tokenize the `base_sentence` column so that it can be used as input to finetune ALBERT
def tokenize_function(examples):
    return tokenizer(examples["base_sentence"], padding="max_length", truncation=True, max_length=512)

tokenized_train_dataset = train_dataset.map(tokenize_function, batched=True)
tokenized_eval_dataset = val_dataset.map(tokenize_function, batched=True)

Map: 100%|██████████| 317/317 [00:00<00:00, 8403.93 examples/s]


In [13]:
# Format the dataset for PyTorch - Remove columns not needed by the model
cols_to_remove = ["Unnamed: 0", "base_sentence", "occupation", "male_sentence", "female_sentence", "stereotypical_gender"]
tokenized_train_dataset = tokenized_train_dataset.remove_columns(cols_to_remove)
tokenized_eval_dataset = tokenized_eval_dataset.remove_columns(cols_to_remove)

# # Rename the 'stereotypical_gender' column to 'labels' (expected by Trainer)
# tokenized_train_dataset = tokenized_train_dataset.rename_column("stereotypical_gender", "labels")
# tokenized_eval_dataset = tokenized_eval_dataset.rename_column("stereotypical_gender", "labels")

# Set format to PyTorch tensors
tokenized_train_dataset.set_format("torch")
tokenized_eval_dataset.set_format("torch")


In [14]:
import evaluate

metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    # Logits are the raw output scores from the model, shape (batch_size, num_labels)
    # Labels are the ground truth, shape (batch_size,)
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [15]:
training_args = TrainingArguments(
    output_dir="./results",             # Directory to save model checkpoints and logs
    num_train_epochs=1,                 # Reduced for quick demonstration; use more epochs (e.g., 3-5) for real tasks
    per_device_train_batch_size=8,      # Adjust based on your GPU memory
    per_device_eval_batch_size=8,       # Adjust based on your GPU memory
    warmup_steps=100,                   # Number of steps for linear warmup
    weight_decay=0.01,                  # Regularization strength
    logging_dir="./logs",               # Directory for TensorBoard logs
    logging_steps=50,                   # Log metrics every 50 steps
    # evaluation_strategy="epoch",        # Evaluate performance at the end of each epoch
    # save_strategy="epoch",              # Save model checkpoint at the end of each epoch
    # load_best_model_at_end=True,        # Load the best model found during training at the end
    metric_for_best_model="accuracy",   # Metric used to determine the best model
    greater_is_better=True,             # Accuracy should be maximized
    report_to="tensorboard",            # Report logs to TensorBoard (can add "wandb" etc.)
    # push_to_hub=False,                # Set to True to push model to Hugging Face Hub
    fp16=torch.cuda.is_available(),     # Use mixed precision training if CUDA is available
)

trainer = Trainer(
    model=model,                        # The model to train (potentially with custom embeddings)
    args=training_args,                 # Training arguments defined above
    train_dataset=tokenized_train_dataset, # Training dataset
    eval_dataset=tokenized_eval_dataset,   # Evaluation dataset
    tokenizer=tokenizer,                # Tokenizer used for data collation (handles padding dynamically if needed)
    compute_metrics=compute_metrics,    # Function to compute evaluation metrics
    # Optional: Data collator can optimize padding
    # data_collator=DataCollatorWithPadding(tokenizer=tokenizer)
)

/tmp/ipykernel_454228/2816268275.py:20: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [16]:
train_result = trainer.train()

trainer.save_model()  # Saves the tokenizer too
trainer.log_metrics("train", train_result.metrics)
trainer.save_metrics("train", train_result.metrics)
trainer.save_state()

# 6. Evaluate the Final Model
print("Evaluating the final model...")
eval_metrics = trainer.evaluate()
print(f"Evaluation Metrics: {eval_metrics}")
trainer.log_metrics("eval", eval_metrics)
trainer.save_metrics("eval", eval_metrics)

Step,Training Loss
50,1.650600
100,0.590400
150,0.420800


/home/bernard/miniconda3/envs/cs4248_project/lib/python3.11/site-packages/transformers/configuration_utils.py:393: UserWarning: Some non-default generation parameters are set in the model config. These should go into either a) `model.generation_config` (as opposed to `model.config`); OR b) a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model).This warning will become an exception in the future.
Non-default generation parameters: {'max_length': 128}
  warnings.warn(
/home/bernard/miniconda3/envs/cs4248_project/lib/python3.11/site-packages/transformers/configuration_utils.py:393: UserWarning: Some non-default generation parameters are set in the model config. These should go into either a) `model.generation_config` (as opposed to `model.config`); OR b) a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model).This warning will b

***** train metrics *****
  epoch                    =        1.0
  total_flos               =    74950GF
  train_loss               =     0.8494
  train_runtime            = 0:00:31.87
  train_samples_per_second =      39.75
  train_steps_per_second   =      4.988
Evaluating the final model...


Evaluation Metrics: {'eval_loss': 0.2789522409439087, 'eval_accuracy': 0.8832807570977917, 'eval_runtime': 2.1046, 'eval_samples_per_second': 150.624, 'eval_steps_per_second': 19.006, 'epoch': 1.0}
***** eval metrics *****
  epoch                   =        1.0
  eval_accuracy           =     0.8833
  eval_loss               =      0.279
  eval_runtime            = 0:00:02.10
  eval_samples_per_second =    150.624
  eval_steps_per_second   =     19.006


In [17]:
# Access the embedding layer again (use the same path as in Step 4)
final_embedding_layer = mobilebert_model.embeddings.word_embeddings

# Get the weights
final_embeddings_tensor = final_embedding_layer.weight.data

# Convert to NumPy if desired (and move to CPU if on GPU)
final_embeddings_numpy = final_embeddings_tensor.cpu().numpy()
print(final_embeddings_numpy.shape)
final_embeddings_numpy

(2016, 128)


array([[-0.01316017,  0.00858272,  0.01150468, ..., -0.01960675,
         0.01102403,  0.01689079],
       [ 0.00010355,  0.00037436, -0.00077101, ..., -0.04639971,
        -0.0490229 , -0.08949148],
       [ 0.03108884, -0.0236288 , -0.02622124, ...,  0.0061381 ,
         0.0333262 , -0.03211844],
       ...,
       [ 0.0199623 ,  0.01472118, -0.02047166, ...,  0.00751895,
        -0.02516552,  0.00245555],
       [-0.01042134,  0.01345772, -0.04290605, ...,  0.01183271,
         0.00020897, -0.01214551],
       [ 0.01525817, -0.04257036, -0.01507506, ...,  0.00786556,
        -0.01782154,  0.00746505]], shape=(2016, 128), dtype=float32)